# Spark Wrangle Exercises
This exercises uses the `case.csv`, `dept.csv`, and `source.csv` files from the san antonio 311 call dataset.
- Read the `case`, `department`, and `source` data into their own spark dataframes.
- Let's see how writing to the local disk works in spark:
    - Write the code necessary to store the source data in both csv and json format, store these as sources_csv and sources_json.
    - Inspect your folder structure. What do you notice?

- Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.  

In [46]:
import pandas as pd

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import StructField, StringType, StringType

In [5]:
spark = SparkSession.builder.getOrCreate()

In [17]:
df_case = spark.read.csv('data/case.csv', header=True, sep=',', inferSchema=True)
df_dept = spark.read.csv('data/dept.csv', header=True, sep=',', inferSchema=True)
df_source = spark.read.csv('data/source.csv', header=True, sep=',', inferSchema=True)

## case.csv

In [30]:
# The dimensions of the case dataset
df_case.count(), len(df_case.columns)

(841704, 14)

In [50]:
# Display the column names
df_case.dtypes

[('case_id', 'int'),
 ('case_opened_date', 'string'),
 ('case_closed_date', 'string'),
 ('SLA_due_date', 'string'),
 ('case_late', 'string'),
 ('num_days_late', 'double'),
 ('case_closed', 'string'),
 ('dept_division', 'string'),
 ('service_request_type', 'string'),
 ('SLA_days', 'double'),
 ('case_status', 'string'),
 ('source_id', 'string'),
 ('request_address', 'string'),
 ('council_district', 'int')]

## dept.csv

In [31]:
# The dimensions of the dept dataset
df_dept.count(), len(df_dept.columns)

(39, 4)

In [51]:
# Display the column names
df_dept.dtypes

[('dept_division', 'string'),
 ('dept_name', 'string'),
 ('standardized_dept_name', 'string'),
 ('dept_subject_to_SLA', 'string')]

## source.csv

In [32]:
# The dimensions of the source dataset
df_source.count(), len(df_source.columns)

(140, 2)

In [52]:
# Display the column names
df_source.dtypes

[('source_id', 'string'), ('source_username', 'string')]

In [53]:
df_source.write.json('data/source_json', mode='overwrite')
df_source.write.csv('data/source_csv', mode='overwrite')

## 1. How old is the latest (in terms of days past SLA) currently open issue? How long has the oldest (in terms of days since opened) currently opened issue been open?

In [55]:
df_case.show()

+----------+----------------+----------------+------------+---------+-------------------+-----------+----------------+--------------------+------------------+-----------+---------+--------------------+----------------+
|   case_id|case_opened_date|case_closed_date|SLA_due_date|case_late|      num_days_late|case_closed|   dept_division|service_request_type|          SLA_days|case_status|source_id|     request_address|council_district|
+----------+----------------+----------------+------------+---------+-------------------+-----------+----------------+--------------------+------------------+-----------+---------+--------------------+----------------+
|1014127332|     1/1/18 0:42|    1/1/18 12:29|9/26/20 0:42|       NO| -998.5087616000001|        YES|Field Operations|        Stray Animal|             999.0|     Closed| svcCRMLS|2315  EL PASO ST,...|               5|
|1014127333|     1/1/18 0:46|     1/3/18 8:11| 1/5/18 8:30|       NO|-2.0126041669999997|        YES|     Storm Water|Remova

## 2. How many Stray Animal cases are there?

## 3. How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?

## 4. Convert the council_district column to a string column.

## 5. Extract the year from the case_closed_date column.

## 6. Convert num_days_late from days to hours in new columns num_hours_late.

## 7. Join the case data with the source and department data.

## 8. Are there any cases that do not have a request source?

## 9. What are the top 10 service request types in terms of number of requests?

## 10. What are the top 10 service request types in terms of average days late?

## 11. Does number of days late depend on department?

## 12. How do number of days late depend on department and request type?